## House Plant Dimensionality Reduction

This notebook will cover the dimensionality reduction performed on the houseplant database (step 3).  

The objective here is to be able to visualise similar plants (e.g. with 2D scatter plots etc...). 

#### Setup

In [1]:
import pandas as pd
import sqlite3
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

DATABASE_LOC = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\PyForFun\House_Plant_Recommender\Database\house_plants.db"

In [2]:
conn = sqlite3.connect(DATABASE_LOC)
c = conn.cursor()
df_features = pd.read_sql_query("SELECT * FROM plant_features", conn)
c.close()
df_features.head()

,Plant_Name,Min_Temp_Degrees_C,Min_Height,Max_Height,Min_Spread,Max_Spread,Max_Height_Capped,Max_Spread_Capped,Sunlight_Ordinal,Watering_Ordinal,Maintenance_Ordinal,Flowers_Ordinal,Type_Bulb,Type_Fern,Type_Herbaceous_perennial,Type_Other,Type_Vine,Color_Not_Colorful,Fruit_Yes
0,Aechmea,-1.1,1.0,3.0,1.0,2.0,3.0,2.0,1,3,2,3,0,0,0,1,0,1,0
1,Ardisia crenata,-12.2,4.0,5.0,4.0,5.0,5.0,5.0,2,3,2,3,0,0,0,0,0,1,1
2,Euphorbia milii,-6.7,3.0,6.0,1.5,3.0,6.0,3.0,4,2,2,3,0,0,0,0,0,1,0
3,Ficus elastica,-1.1,50.0,100.0,50.0,100.0,20.0,20.0,1,3,1,2,0,0,0,0,0,1,0
4,Woodsia obtusa,-34.4,1.0,1.5,2.0,2.5,1.5,2.5,2,3,2,1,0,1,0,0,0,1,0


#### Dimensionality Reduction 
need to do some scaling and then time to do some projections! 
t-sne?